# 11. 심층 신경망 훈련하기

In [2]:
# 공통 모듈 임포트
import numpy as np
import os

# 깔금한 그래프 출력을 위해
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)    
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)
plt.rc('font', family='AppleGothic')
plt.rcParams['axes.unicode_minus'] = False

## 11.1 그레이디언트 소실과 폭주 문제

### 11.1.1 글로럿과 He 초기화

In [3]:
from tensorflow import keras
import tensorflow as tf

keras.layers.Dense(10, activation='relu', kernel_initializer='he_normal')

### 11.1.2 수렴하지 않는 활성화 함수

In [5]:
keras.layers.Dense(10, kernel_initializer='he_normal')
keras.layers.LeakyReLU(alpha=0.2)

In [6]:
layer = keras.layers.Dense(10, activation='selu', kernel_initializer='lecun_normal')

### 11.1.3 배치 정규화

In [7]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape = [28, 28]),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300, activation='elu', kernel_initializer='he_normal'),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(100, activation='elu', kernel_initializer='he_normal'),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(10, activation='softmax')
])

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 batch_normalization (Batch  (None, 784)               3136      
 Normalization)                                                  
                                                                 
 dense_3 (Dense)             (None, 300)               235500    
                                                                 
 batch_normalization_1 (Bat  (None, 300)               1200      
 chNormalization)                                                
                                                                 
 dense_4 (Dense)             (None, 100)               30100     
                                                                 
 batch_normalization_2 (Bat  (None, 100)               4

In [9]:
[(var.name, var.trainable) for var in model.layers[1].variables]

[('batch_normalization/gamma:0', True),
 ('batch_normalization/beta:0', True),
 ('batch_normalization/moving_mean:0', False),
 ('batch_normalization/moving_variance:0', False)]

In [10]:
model.layers[1].updates

/var/folders/jd/_tlvgz1d22x3k8jpbtplqcsh0000gn/T/ipykernel_69348/3873162892.py:1: UserWarning: `layer.updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  model.layers[1].updates


[]

In [11]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300, kernel_initializer='he_normal', use_bias=False),
    keras.layers.BatchNormalization(),
    keras.layers.Activation('elu'),
    keras.layers.Dense(100, kernel_initializer='he_normal', use_bias=False),
    keras.layers.BatchNormalization(),
    keras.layers.Activation('elu'),
    keras.layers.Dense(10, activation='softmax')
])

### 11.1.4 그레이디언트 클리핑

In [12]:
optimizer = keras.optimizers.SGD(clipvalue=1.0)
model.compile(loss='mse', optimizer=optimizer)

## 11.2 사전훈련된 층 재사용하기

### 11.2.1 케라스를 사용한 전이 학습

In [13]:
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()
X_train_full = X_train_full / 255.0
X_test = X_test / 255.0
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

In [15]:
def split_dataset(X, y):
    y_5_or_6 = (y == 5) | (y == 6) # sandals or shirts
    y_A = y[~y_5_or_6]
    y_A[y_A > 6] -= 2 # class indices 7, 8, 9 should be moved to 5, 6, 7
    y_B = (y[y_5_or_6] == 6).astype(np.float32) # binary classification task: is it a shirt (class 6)?
    return ((X[~y_5_or_6], y_A),
            (X[y_5_or_6], y_B))

(X_train_A, y_train_A), (X_train_B, y_train_B) = split_dataset(X_train, y_train)
(X_valid_A, y_valid_A), (X_valid_B, y_valid_B) = split_dataset(X_valid, y_valid)
(X_test_A, y_test_A), (X_test_B, y_test_B) = split_dataset(X_test, y_test)
X_train_B = X_train_B[:200]
y_train_B = y_train_B[:200]

In [18]:
tf.random.set_seed(42)
np.random.seed(42)

In [19]:
model_A = keras.models.Sequential()
model_A.add(keras.layers.Flatten(input_shape=[28, 28]))
for n_hidden in (300, 100, 50, 50, 50):
    model_A.add(keras.layers.Dense(n_hidden, activation="selu"))
model_A.add(keras.layers.Dense(8, activation="softmax"))

In [20]:
model_A.compile(loss="sparse_categorical_crossentropy",
                optimizer=keras.optimizers.SGD(learning_rate=1e-3),
                metrics=["accuracy"])

In [21]:
history = model_A.fit(X_train_A, y_train_A, epochs=20,
                    validation_data=(X_valid_A, y_valid_A))

Epoch 1/20
1375/1375 [==============================] - 2s 1ms/step - loss: 0.5611 - accuracy: 0.8215 - val_loss: 0.3784 - val_accuracy: 0.8727
Epoch 2/20
1375/1375 [==============================] - 2s 1ms/step - loss: 0.3452 - accuracy: 0.8844 - val_loss: 0.3185 - val_accuracy: 0.8926
Epoch 3/20
1375/1375 [==============================] - 2s 1ms/step - loss: 0.3098 - accuracy: 0.8948 - val_loss: 0.2916 - val_accuracy: 0.9008
Epoch 4/20
1375/1375 [==============================] - 2s 1ms/step - loss: 0.2913 - accuracy: 0.9017 - val_loss: 0.2800 - val_accuracy: 0.9076
Epoch 5/20
1375/1375 [==============================] - 2s 1ms/step - loss: 0.2788 - accuracy: 0.9057 - val_loss: 0.2694 - val_accuracy: 0.9108
Epoch 6/20
1375/1375 [==============================] - 2s 1ms/step - loss: 0.2694 - accuracy: 0.9085 - val_loss: 0.2683 - val_accuracy: 0.9053
Epoch 7/20
1375/1375 [==============================] - 2s 1ms/step - loss: 0.2616 - accuracy: 0.9113 - val_loss: 0.2647 - val_accuracy:

In [22]:
model_A.save("my_model_A.h5")

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [25]:
model_A_clone = keras.models.clone_model(model_A)
model_A_clone.set_weights(model_A.get_weights())
model_B_on_A = keras.models.Sequential(model_A_clone.layers[:-1])
model_B_on_A.add(keras.layers.Dense(1, activation="sigmoid"))

In [26]:
for layer in model_B_on_A.layers[:-1]:
    layer.trainable = False

model_B_on_A.compile(loss='binary_crossentropy', optimizer='sgd', metrics=['accuracy'])

In [28]:
history = model_B_on_A.fit(X_train_B, y_train_B, epochs=4, validation_data=(X_valid_B, y_valid_B))

for layer in model_B_on_A.layers[:-1]:
    layer.trainable = True

optimizer = keras.optimizers.SGD(lr = 1e-4)

model_B_on_A.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

history = model_B_on_A.fit(X_train_B, y_train_B, epochs=16, validation_data=(X_valid_B, y_valid_B))

Epoch 1/4
7/7 [==============================] - 0s 20ms/step - loss: 0.5086 - accuracy: 0.7250 - val_loss: 0.4305 - val_accuracy: 0.7708
Epoch 2/4
7/7 [==============================] - 0s 6ms/step - loss: 0.3189 - accuracy: 0.8550 - val_loss: 0.3001 - val_accuracy: 0.8682
Epoch 3/4
7/7 [==============================] - 0s 6ms/step - loss: 0.2244 - accuracy: 0.9400 - val_loss: 0.2298 - val_accuracy: 0.9158
Epoch 4/4
7/7 [==============================] - 0s 6ms/step - loss: 0.1724 - accuracy: 0.9700 - val_loss: 0.1853 - val_accuracy: 0.9412


Epoch 1/16
7/7 [==============================] - 0s 20ms/step - loss: 0.0927 - accuracy: 0.9850 - val_loss: 0.0702 - val_accuracy: 0.9817
Epoch 2/16
7/7 [==============================] - 0s 8ms/step - loss: 0.0411 - accuracy: 0.9900 - val_loss: 0.0542 - val_accuracy: 0.9848
Epoch 3/16
7/7 [==============================] - 0s 11ms/step - loss: 0.0290 - accuracy: 1.0000 - val_loss: 0.0465 - val_accuracy: 0.9858
Epoch 4/16
7/7 [==============================] - 0s 8ms/step - loss: 0.0227 - accuracy: 1.0000 - val_loss: 0.0420 - val_accuracy: 0.9848
Epoch 5/16
7/7 [==============================] - 0s 7ms/step - loss: 0.0183 - accuracy: 1.0000 - val_loss: 0.0393 - val_accuracy: 0.9858
Epoch 6/16
7/7 [==============================] - 0s 7ms/step - loss: 0.0155 - accuracy: 1.0000 - val_loss: 0.0374 - val_accuracy: 0.9868
Epoch 7/16
7/7 [==============================] - 0s 7ms/step - loss: 0.0134 - accuracy: 1.0000 - val_loss: 0.0375 - val_accuracy: 0.9848
Epoch 8/16
7/7 [================

In [29]:
model_B_on_A.evaluate(X_test_B, y_test_B)

63/63 [==============================] - 0s 774us/step - loss: 0.0164 - accuracy: 0.9950


[0.01644645445048809, 0.9950000047683716]

### 11.2.2 비지도 사전훈련

### 11.2.3 보조 작업에서 사전훈련

## 11.3 고속 옵테마이저

### 11.3.1 모멘텀 최적화

In [4]:
optimizer = keras.optimizers.SGD(lr=0.001, momentum=0.9)

### 11.3.2 네스테로프 가속 경사

In [5]:
optimizer = keras.optimizers.SGD(lr=0.001, momentum=0.9, nesterov=True)

### 11.3.3 AdaGrad

### 11.3.4 RMSProp

In [6]:
optimizer = keras.optimizers.RMSprop(lr=0.001, rho=0.9)

### 11.3.5 Adam과 Nadam 최적화

In [7]:
optimizer = keras.optimizers.Adam(lr=0.001, beta_1 = 0.9, beta_2=0.999)

### 11.3.6 학습률 스케줄링

## 11.4 규제를 사용해 과대적합 피하기